In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
import os

from tensorflow.keras.optimizers import Adam
os.environ["CUDA_VISIBLE_DEVICES"] = '2'
# 加载 CIFAR-10 数据
import random
import numpy as np
# Load and preprocess CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


2024-07-22 16:09:07.000604: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
##该模型并没有使用双重对齐，因为我使用的数据错了。

In [3]:
from torchvision import transforms
import imageio
import numpy as np
from PIL import Image
from random import sample
# 定义将NumPy数组转换为float32的类
class convertNumpyArrayToFloat32(object):
    def __call__(self, np_img):
        return np_img.astype(np.float32)

# 定义将NumPy数组的像素值裁剪到[0, 255]范围内并转换为uint8的类
class clipAndConvertNumpyArrayToUint8(object):
    def __call__(self, np_img):
        return np.clip(np_img, 0, 255).astype(np.uint8)

# 定义Blended Injection攻击的类
class blendedImageAttack(object):
    def __init__(self, target_image, blended_rate):
        self.target_image = target_image
        self.blended_rate = blended_rate

    def __call__(self, img):
        return (1-self.blended_rate) * img + (self.blended_rate) * self.target_image

# 定义一个通用的数据增强组合类
class general_compose(object):
    def __init__(self, transform_list):
        self.transform_list = transform_list

    def __call__(self, img, *args, **kwargs):
        for transform, if_all in self.transform_list:
            if if_all == False:
                img = transform(img)
            else:
                img = transform(img, *args, **kwargs)
        return img

# 定义一个转换管道，用于将图像转换为适合攻击的格式
trans = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((32, 32)),  # 调整图像大小
    transforms.ToTensor()
])

# 读取用于攻击的目标图像
target_image_path = '/home/xipeng/BackDOOR/Blended/hello_kitty.jpeg'  # 请替换为你的目标图像的路径
trans = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((32, 32)),  # 调整图像大小
    transforms.ToTensor()
])
target_image = trans(imageio.imread(target_image_path)).numpy().transpose(1, 2, 0) * 255
# 现在你可以使用train_bd_transform和test_bd_transform来创建你的攻击样本
train_bd_transform = general_compose([
    (lambda x: Image.fromarray((x * 255).astype('uint8')), False),  # 新增：将NumPy数组转换为PIL图像
    (transforms.Resize((32, 32)), False),
    (np.array, False),
    (blendedImageAttack(target_image, 0.2), True),
    (convertNumpyArrayToFloat32(), False),
    (clipAndConvertNumpyArrayToUint8(), False),
])

test_bd_transform = general_compose([
    (lambda x: Image.fromarray((x * 255).astype('uint8')), False),  # 新增：将NumPy数组转换为PIL图像
    (transforms.Resize((32, 32)), False),
    (np.array, False),
    (blendedImageAttack(target_image, 0.2), True),
    (convertNumpyArrayToFloat32(), False),
    (clipAndConvertNumpyArrayToUint8(), False),
])





    
    
# 选择1%的训练数据进行攻击
num_samples = int(0.2 * x_val.shape[0])
indices = np.random.choice(y_val.shape[0], num_samples, replace=False)
target_class = 1  # 目标类别
num_classes = 10  # CIFAR-10 数据集的类别数
x_val_tigger = x_val.copy()
y_val_tigger = y_val.copy()

# 新的数据集和标签列表
x_val_triggered = []  # 存储触发器转换后的图像
y_val_triggered = []  # 存储更新后的标签

# 应用攻击到选中的图像
for i in indices:
    test_image = x_val_tigger[i]  # 获取单个图像
    test_image_1 = train_bd_transform(test_image)  # 应用转换
    x_val_tigger[i] = test_image_1.astype('float32') / 255.0  # 归一化
    y_val_tigger[i] = to_categorical(target_class, num_classes=num_classes)  # 更新标签
    
    x_val_triggered.append(x_val_tigger[i])
    y_val_triggered.append(to_categorical(target_class, num_classes=num_classes))

# 将列表转换为NumPy数组（如果需要）
x_val_triggered = np.array(x_val_triggered)
y_val_triggered = np.array(y_val_triggered)
    
    
    
    
    
x_test_tigger = x_test.copy()
y_test_tigger = y_test.copy()

for i in range(len(x_test_tigger)):
    index = i 
    test_image = x_test_tigger[index]  # 获取单个图像
    test_image_1 = train_bd_transform(test_image)  # 应用转换
    x_test_tigger[index] = test_image_1.astype('float32') / 255.0  # 归一化
    y_test_tigger[index] = to_categorical(target_class, num_classes=num_classes)  # 更新标签
    

/home/xipeng/anaconda3/envs/keras2/lib/python3.7/site-packages/ipykernel_launcher.py:52: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.


In [4]:
##进行攻击的思路
#1.将数据集分成两部分  8比2吧
#2.每个epcoch后再单独优化一下服务器模型，保持模拟的客户端模型不变，后门数据要多一些
#3.验证数据集结果

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models



def create_client_model(input_shape):
    input_layer = layers.Input(shape=input_shape)
    
    # 第一层卷积
    x = layers.Conv2D(32, 3, strides=1, padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # 第二层卷积
    x = layers.Conv2D(64, 3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # 新增的第三层卷积
    x = layers.Conv2D(128, 3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    model = models.Model(inputs=input_layer, outputs=x)
    return model


In [6]:
from tensorflow.keras import layers, models

def res_block(filters, strides):
    def block(x):
        shortcut = x

        x = layers.Conv2D(filters, 3, padding='same', strides=strides)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)

        x = layers.Conv2D(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)

        # 捷径连接前的调整
        shortcut = layers.Conv2D(filters, 1, strides=strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

        x = layers.add([x, shortcut])
        x = layers.Activation('relu')(x)
        return x
    return block

def create_server_model():
    # 调整输入层的定义以匹配修改后的客户端模型的输出
    input_layer = layers.Input(shape=(32, 32, 128))  # 注意这里的变化

    # 残差块定义保持不变，继续使用提前定义的 res_block
    x = res_block(128, 1)(input_layer)  # 使用第一个残差块
    x = res_block(128, 2)(x)            # 使用第二个残差块
    x = res_block(128, 2)(x)           # 使用第三个残差块
    x = res_block(256, 2)(x)           # 使用第四个残差块

    # 全局平均池化和输出层保持不变
    x = layers.GlobalAveragePooling2D()(x)
    output = layers.Dense(10, activation='softmax')(x)
    
    model = models.Model(inputs=input_layer, outputs=output)
    return model





In [7]:
# 创建客户端模型
client_model = create_client_model(input_shape=(32, 32, 3))

# 编译客户端模型
client_model.compile(optimizer=Adam(),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])


2024-07-22 16:09:32.020870: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2024-07-22 16:09:32.103703: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:af:00.0 name: NVIDIA Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2024-07-22 16:09:32.103772: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2024-07-22 16:09:32.111174: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2024-07-22 16:09:32.120574: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2024-07-22 16:09:32.121235: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcu

In [8]:
client_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)       

In [9]:
# 以下是服务器模型训练的简化示例
server_model = create_server_model()
server_model.compile(optimizer=Adam(),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

In [10]:
##创建三个影子模型，分别是1层卷积，2层卷积核3层卷积
server_model_1 = create_server_model()
server_model_1.compile(optimizer=Adam(),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

In [11]:
# Assuming TensorFlow is installed and the functions are defined in your script
input_shape = (32, 32, 3)

client_model_1 = create_client_model(input_shape=(32, 32, 3))

# You can then print the model summaries to verify their structures
print(client_model_1.summary())




Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_27 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
activation_19 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_28 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
activation_20 (Activation)   (None, 32, 32, 64)       

In [12]:
@tf.function
def train_step_client(x, y):
    # 使用客户端和服务器模型进行训练
    with tf.GradientTape() as tape_client, tf.GradientTape() as tape_server:
        # 客户端模型前向传播
        client_outputs = client_model(x, training=True)
        # 服务器模型前向传播
        server_logits = server_model(client_outputs, training=True)
        # 计算损失
        loss = loss_fn(y, server_logits)
    
    # 计算并应用服务器模型梯度
    grads_server = tape_server.gradient(loss, server_model.trainable_variables)
    optimizer_server.apply_gradients(zip(grads_server, server_model.trainable_variables))
    
    # 计算并应用客户端模型梯度
    grads_client = tape_client.gradient(loss, client_model.trainable_variables)
    optimizer_client.apply_gradients(zip(grads_client, client_model.trainable_variables))

    return loss


In [13]:
def train_step_client_1(x, y):
    # 使用客户端和服务器模型进行训练
    with tf.GradientTape() as tape_client, tf.GradientTape() as tape_server:
        # 客户端模型前向传播
        client_outputs = client_model_1(x, training=True)
        # 服务器模型前向传播
        server_logits = server_model_1(client_outputs, training=True)
        # 计算损失
        loss = loss_fn_1(y, server_logits)
    
    # 计算并应用服务器模型梯度
    grads_server = tape_server.gradient(loss, server_model_1.trainable_variables)
    optimizer_server_1.apply_gradients(zip(grads_server, server_model_1.trainable_variables))
    
    # 计算并应用客户端模型梯度
    grads_client = tape_client.gradient(loss, client_model_1.trainable_variables)
    optimizer_client_1.apply_gradients(zip(grads_client, client_model_1.trainable_variables))

    return loss

In [14]:

optimizer_client = tf.keras.optimizers.Adam()
optimizer_server = tf.keras.optimizers.Adam()
optimizer_client_1 = tf.keras.optimizers.Adam()
optimizer_server_1 = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
loss_fn_1 = tf.keras.losses.CategoricalCrossentropy(from_logits=True)


batch_size = 64
def augment(image, label):
    # 随机水平翻转图像
    image = tf.image.random_flip_left_right(image)
    # 随机调整亮度
    image = tf.image.random_brightness(image, max_delta=0.1)
    # 确保图像值仍然在0到1的范围内
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image, label

# 假设x_train, y_train, x_val, y_val, x_val_tigger, y_val_tigger已经定义并准备好了

# 训练数据集 - 应用数据增强
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.map(augment).shuffle(len(x_train)).batch(batch_size)

# 客户端验证数据集 - 也应用数据增强
client_1_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
client_1_dataset = client_1_dataset.map(augment).shuffle(len(x_val)).batch(batch_size)

# 服务器训练数据集 - 也应用数据增强


In [15]:
# 初始化TensorBoard回调
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
epochs = 60


2024-07-22 16:09:43.234715: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2024-07-22 16:09:43.234850: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1391] Profiler found 1 GPUs
2024-07-22 16:09:43.236993: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcupti.so.10.1
2024-07-22 16:09:43.242026: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1441] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES


In [16]:

for epoch in range(50):
    for x_batch, y_batch in train_dataset:
        loss = train_step_client(x_batch, y_batch)
    print(f"  Client Model Loss: {loss.numpy():.4f}")

2024-07-22 16:09:56.218960: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2024-07-22 16:09:57.060762: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7
2024-07-22 16:10:01.663798: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2024-07-22 16:10:02.267956: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


  Client Model Loss: 1.8284
  Client Model Loss: 1.7625
  Client Model Loss: 1.7541
  Client Model Loss: 1.7734
  Client Model Loss: 1.8187
  Client Model Loss: 1.7242
  Client Model Loss: 1.7772
  Client Model Loss: 1.6179
  Client Model Loss: 1.6943
  Client Model Loss: 1.7025
  Client Model Loss: 1.5815
  Client Model Loss: 1.6203
  Client Model Loss: 1.6530
  Client Model Loss: 1.6389
  Client Model Loss: 1.5871
  Client Model Loss: 1.5575
  Client Model Loss: 1.6120
  Client Model Loss: 1.5831
  Client Model Loss: 1.5828
  Client Model Loss: 1.5944
  Client Model Loss: 1.5666
  Client Model Loss: 1.5532
  Client Model Loss: 1.6034
  Client Model Loss: 1.5728
  Client Model Loss: 1.5899
  Client Model Loss: 1.6223
  Client Model Loss: 1.5786
  Client Model Loss: 1.6021
  Client Model Loss: 1.5447
  Client Model Loss: 1.5030
  Client Model Loss: 1.5310
  Client Model Loss: 1.4963
  Client Model Loss: 1.5173
  Client Model Loss: 1.5228
  Client Model Loss: 1.6075
  Client Model Loss:

In [17]:

for epoch in range(50):
    for x_batch, y_batch in client_1_dataset:   ##只修改这部分
        loss_1 = train_step_client_1(x_batch, y_batch)
    print(f"  Client Model 1 Loss: {loss_1.numpy():.4f}")

  Client Model 1 Loss: 2.0910
  Client Model 1 Loss: 2.0872
  Client Model 1 Loss: 2.0502
  Client Model 1 Loss: 1.8106
  Client Model 1 Loss: 1.8965
  Client Model 1 Loss: 1.8406
  Client Model 1 Loss: 1.7798
  Client Model 1 Loss: 1.8871
  Client Model 1 Loss: 1.6805
  Client Model 1 Loss: 1.9170
  Client Model 1 Loss: 1.9774
  Client Model 1 Loss: 1.7053
  Client Model 1 Loss: 1.7007
  Client Model 1 Loss: 1.8655
  Client Model 1 Loss: 1.8575
  Client Model 1 Loss: 1.8176
  Client Model 1 Loss: 1.6787
  Client Model 1 Loss: 1.7499
  Client Model 1 Loss: 1.6411
  Client Model 1 Loss: 1.7048
  Client Model 1 Loss: 1.8134
  Client Model 1 Loss: 1.8652
  Client Model 1 Loss: 1.9078
  Client Model 1 Loss: 1.7046
  Client Model 1 Loss: 1.6913
  Client Model 1 Loss: 1.7109
  Client Model 1 Loss: 1.6575
  Client Model 1 Loss: 1.6791
  Client Model 1 Loss: 1.7071
  Client Model 1 Loss: 1.6703
  Client Model 1 Loss: 1.5733
  Client Model 1 Loss: 1.5747
  Client Model 1 Loss: 1.5867
  Client M

In [18]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
def create_combined_model(client_model, server_model, compile_only=False):
    # Getting the input of the client model
    client_input = client_model.input
    
    # Getting the intermediate output by passing the input through the client model
    client_output = client_model(client_input)
    
    # The client model's output is used as the input for the server model
    server_output = server_model(client_output)
    
    # Defining a new model that chains the client and server models
    combined_model = Model(inputs=client_input, outputs=server_output)
    
    # Compile the combined model
    combined_model.compile(optimizer=Adam(),
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])
    
    if not compile_only:
        # If not compile_only, evaluate the model
        loss, accuracy = combined_model.evaluate(x_test, y_test, verbose=0)
        print(f"Test loss: {loss}, Test accuracy: {accuracy}")
    
    return combined_model


In [19]:


combined_model = create_combined_model(client_model, server_model, compile_only=True)
combined_model_1 = create_combined_model(client_model_1, server_model_1, compile_only=True)

In [20]:
loss, accuracy = combined_model.evaluate(x_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")


313/313 [==============================] - 7s 21ms/step - loss: 1.7387 - accuracy: 0.8319
Test loss: 1.7386630773544312, Test accuracy: 0.8319000005722046


In [21]:
loss, accuracy = combined_model_1.evaluate(x_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

313/313 [==============================] - 2s 7ms/step - loss: 3.4322 - accuracy: 0.6541
Test loss: 3.432222366333008, Test accuracy: 0.6541000008583069


In [22]:
loss, accuracy = combined_model.evaluate(x_test_tigger, y_test_tigger)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")


313/313 [==============================] - 2s 6ms/step - loss: 29.0880 - accuracy: 0.0809
Test loss: 29.087982177734375, Test accuracy: 0.08089999854564667


In [23]:
class Autoencoder(tf.keras.Model):
    def __init__(self, input_shape):
        super(Autoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=input_shape),
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=2),
            tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)
        ])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
            tf.keras.layers.Conv2DTranspose(32, kernel_size=3, strides=2, activation='relu', padding='same'),
            tf.keras.layers.Conv2D(128, (3, 3), activation='sigmoid', padding='same')
        ])

    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Autoencoder(input_shape=(32, 32, 128))  # 修改这里以符合实际输入维度


In [24]:
def train_step(x_batch, x_batch_1, client_model, client_model_1, autoencoder, optimizer):
    with tf.GradientTape() as tape:
        client_output = client_model(x_batch, training=True)
        client_output_1 = client_model_1(x_batch_1, training=True)
        reconstructed_output = autoencoder(client_output, training=True)
        loss = tf.reduce_mean(tf.keras.losses.mean_squared_error(client_output_1, reconstructed_output))
    gradients = tape.gradient(loss, autoencoder.trainable_variables + client_model.trainable_variables + client_model_1.trainable_variables)
    optimizer.apply_gradients(zip(gradients, autoencoder.trainable_variables + client_model.trainable_variables + client_model_1.trainable_variables))

    return loss


In [25]:
x_train_1 = np.tile(x_val_tigger, (4, 1, 1, 1)) 

In [26]:
train_dataset_2 = tf.data.Dataset.from_tensor_slices((x_test, x_test_tigger))
train_dataset_2 = train_dataset_2.shuffle(buffer_size=10000).batch(batch_size)  # 这里的10000是缓冲区大小
epochs = 20
batch_size = 32
optimizer = tf.keras.optimizers.Adam()

In [27]:
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for (x_batch, x_batch_1) in train_dataset_2:
        # 在这里调用训练步骤
        # 确保检查x_batch和x_batch_1的维度是否符合模型的输入要求    
        # 假设 train_step 是您的训练函数
        loss = train_step(x_batch, x_batch_1, client_model, client_model_1, autoencoder, optimizer)
    print(loss)

Epoch 1/20
tf.Tensor(0.002094169, shape=(), dtype=float32)
Epoch 2/20
tf.Tensor(0.0021462631, shape=(), dtype=float32)
Epoch 3/20
tf.Tensor(0.0015353761, shape=(), dtype=float32)
Epoch 4/20
tf.Tensor(0.00058187125, shape=(), dtype=float32)
Epoch 5/20
tf.Tensor(0.00155226, shape=(), dtype=float32)
Epoch 6/20
tf.Tensor(0.0006288801, shape=(), dtype=float32)
Epoch 7/20
tf.Tensor(0.00054512895, shape=(), dtype=float32)
Epoch 8/20
tf.Tensor(0.00041250233, shape=(), dtype=float32)
Epoch 9/20
tf.Tensor(0.0011568337, shape=(), dtype=float32)
Epoch 10/20
tf.Tensor(0.000436031, shape=(), dtype=float32)
Epoch 11/20
tf.Tensor(0.00018584766, shape=(), dtype=float32)
Epoch 12/20
tf.Tensor(0.00014696032, shape=(), dtype=float32)
Epoch 13/20
tf.Tensor(0.00010717621, shape=(), dtype=float32)
Epoch 14/20
tf.Tensor(0.0004982742, shape=(), dtype=float32)
Epoch 15/20
tf.Tensor(9.982063e-05, shape=(), dtype=float32)
Epoch 16/20
tf.Tensor(0.00017883745, shape=(), dtype=float32)
Epoch 17/20
tf.Tensor(0.000763

In [28]:
##后续正常模型的训练，直接微调
##自编码器只正常传播，不反向。


In [29]:
@tf.function
def train_step_2(data, labels):
    with tf.GradientTape(persistent=True) as tape:
        # 前向传播
        client_outputs = client_model(data, training=True)
        encoded_decoded = autoencoder(client_outputs, training=False)  # 训练时不更新autoencoder
        predictions = server_model(encoded_decoded, training=True)
        
        # 计算损失
        loss = tf.keras.losses.categorical_crossentropy(labels, predictions)

    # 计算梯度
    gradients_client = tape.gradient(loss, client_model.trainable_variables)
    gradients_server = tape.gradient(loss, server_model.trainable_variables)
    
    # 更新权重
    optimizer.apply_gradients(zip(gradients_client, client_model.trainable_variables))
    optimizer.apply_gradients(zip(gradients_server, server_model.trainable_variables))
    
    return loss

# 例如的优化器
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)


In [30]:
# 数据集和批次大小设定
batch_size = 32
epochs = 5
#(x_train, y_train), _ = tf.keras.datasets.cifar10.load_data()
#y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)

# 直接在循环中使用train_step
for epoch in range(epochs):
    for x_batch, y_batch in train_dataset:
        loss = train_step_2(x_batch, y_batch)
    print("Epoch: {}, Loss: {}".format(epoch, loss.numpy()))

Epoch: 0, Loss: [0.30212745 0.8191131  1.8174973  0.4317621  0.3265142  2.0264502
 0.30242208 0.31015816 4.1243925  0.70398325 1.0287144  3.9378228
 0.30984473 2.5135634  0.30994108 1.6744096  0.47238195 1.7562135
 0.6449893  3.1963816  1.2983826  1.5611374  0.36936328 0.33566245
 2.3561134  0.9865504  4.518546   2.7013977  2.2468638  2.1425412
 0.6374242  1.1514287 ]
Epoch: 1, Loss: [0.29312965 0.6680321  2.181323   0.665538   0.36440378 1.7536446
 0.38533184 0.19494513 3.4144745  0.7334363  0.6483623  5.7275343
 0.21628185 1.89905    0.34990227 1.2374239  0.33803698 1.778122
 0.8422676  2.4373097  1.2187704  1.3273392  0.43295467 0.4396556
 2.555488   0.67564815 2.8291912  2.0836923  1.5477829  1.2549945
 1.0343094  0.82733035]
Epoch: 2, Loss: [0.36679232 0.4307565  2.450375   1.090271   0.3562171  1.6845193
 0.36138275 0.17217034 2.977985   0.7217267  0.54260445 6.4156027
 0.16327074 1.9236842  0.40977895 1.036258   0.25697273 1.6139234
 0.7292527  1.6054527  0.9771038  1.3069988  0

In [31]:
loss, accuracy = combined_model.evaluate(x_test_tigger, y_test_tigger)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")


313/313 [==============================] - 2s 6ms/step - loss: 252.2959 - accuracy: 0.4497
Test loss: 252.2959442138672, Test accuracy: 0.4496999979019165


In [32]:
loss, accuracy = combined_model.evaluate(x_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

313/313 [==============================] - 2s 6ms/step - loss: 851.8169 - accuracy: 0.1237
Test loss: 851.81689453125, Test accuracy: 0.12370000034570694


In [33]:
#313/313 [==============================] - 1s 3ms/step - loss: 0.9457 - accuracy: 0.7466
#Test loss: 0.9456651210784912, Test accuracy: 0.7465999722480774


## 